In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "src"))

In [61]:
import datetime
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from copy import deepcopy
import seaborn.objects as so
import plotly.express as px
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots


from variables import get_db_path
from db_commands.connect import connect_to_real_db 
from db_commands.read import read_all_entries, aggregate_focus_project_date


In [6]:
connection = connect_to_real_db(get_db_path())
df = read_all_entries(connection)
df
# df_agg = aggregate_focus_project_date(df)
# df_agg = df_agg.group_by("focus_area", "date").agg([pl.col("elapsed_time").sum(), pl.col("project")])
# df_agg

entry_id,date,focus_area,project,description,minutes,activity_type,elapsed_time
i64,str,str,str,str,i64,str,duration[ms]
1,"""2024-12-11""","""personal_code""","""time_logger""","""creating db and commands""",33,"""coding""",33m
1,"""2024-12-11""","""personal_code""","""time_logger""","""creating db and commands""",76,"""coding""",1h 16m
2,"""2024-12-05""","""personal_code""","""time_logger""","""setup""",55,"""coding""",55m
2,"""2024-12-05""","""personal_code""","""time_logger""","""setup""",84,"""coding""",1h 24m
3,"""2024-12-06""","""career""","""building africa's cities summi…","""mel lane grant app""",32,"""writing""",32m
…,…,…,…,…,…,…,…
10,"""2024-12-14""","""admin""","""cee243""","""grading project 3""",48,"""grading""",48m
10,"""2024-12-14""","""admin""","""cee243""","""grading project 3""",71,"""grading""",1h 11m
10,"""2024-12-14""","""admin""","""cee243""","""grading project 3""",51,"""grading""",51m


In [14]:
grouped_df = df.group_by("focus_area", "project").agg(pl.col("elapsed_time").sum()).sort(by="focus_area")
grouped_df

focus_area,project,elapsed_time
str,str,duration[ms]
"""admin""","""cee243""",3h 17m
"""career""","""summer24 job application""",6h 16m
"""career""","""building africa's cities summi…",1h 32m
"""foundations""","""probabilistic graphical models""",1h 30m
"""personal_code""","""time_logger""",4h 8m
"""personal_code""","""slowcial""",3h 42m
"""phd""","""SERC""",32m


In [36]:
type(grouped_df[0]["elapsed_time"][0])

datetime.timedelta

In [56]:
np.full((2,3), "hi").tolist()

[['hi', 'hi', 'hi'], ['hi', 'hi', 'hi']]

In [64]:
def calc_nrows(nplots, ncols=2):
    return nplots // ncols + (nplots % ncols > 0)

def make_indices(nrows, ncols):
    range_rows = list(range(1, nrows + 1))
    range_cols = list(range(1, ncols + 1))
    return list(product(range_rows, range_cols))

def prep_subplot(nplots, ncols=2):
    d = {}
    d["cols"] = ncols
    d["rows"] = calc_nrows(nplots, ncols)
    d["specs"] = np.full((d["rows"], d["cols"]), {"type": "domain"}).tolist()
    indices = make_indices(d["rows"], d["cols"])
    return d, indices

In [63]:
make_indices(2,3)

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]

In [65]:
n_focus_areas = grouped_df["focus_area"].n_unique()
ncols = 3
subplot_details, indices =prep_subplot(n_focus_areas, ncols)
fig = make_subplots(**subplot_details,
    subplot_titles=[name[0] for name, _ in grouped_df.group_by("focus_area")]
    # TODO total time for focus area, and maybe percent of time spent here?
)

for ix, (name, data) in zip(indices, grouped_df.group_by("focus_area")):
    fig.add_trace(    
        go.Pie(
            labels=data["project"].to_list(),
            values=data["elapsed_time"].cast(int).to_list(),
            name = name[0]
        ), 
        ix[0], ix[1]

    )

fig.update_traces(showlegend=False)

fig.show()

In [27]:
for name, data in grouped_df.group_by("focus_area"):
    print(name[0])
    # print(data["project"].to_list()
    #       )
    # # print(name, data.select(pl.col("project"), pl.col("elapsed_time")))

personal_code
phd
admin
career
foundations


In [ ]:
# subplot for each focus area.. 
# labels are the project worked on, values are the time worked on...
# name is the focus area
# get descriptions in there later, maybe using the suburst chart... 